### 4.5 학습 알고리즘 구현하기

신경망 학습의 절차를 복습해보자:  
전체) 신경망에는 적용가능한 가중치와 편향이 있고, 이들을 훈련 데이터에 적응하도록 조정하는 과정을 학습이라 한다.  

1) 미니배치  
훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하여 그 손실 함수 값을 줄이는 것이 목표이다.  

2) 기울기 산출  
미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다.  

3) 매개변수 갱신  
가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.  

4) 반복
1 ~ 3단계를 반복한다.  

이때 무작위로 미니배치를 선정하기에 확률적 경사 하강법 (sochastic gradient descent)라고 부른다. 

#### 4.5.1 2층 신경망 클래스 구현하기

In [10]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    # 초기화를 수행한다.
    # 인수는 입력층, 은닉층, 출력층의 뉴런 수
    def __init__(self, input_size, hidden_size, output_size,
                weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}    # 매개변수를 저장할 딕셔너리 변수 (인스턴스 변수)
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)     # 첫번째 층의 가중치
        self.params['b1'] = np.zeros(hidden_size)   # 첫번째 층의 편향
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)    # 두번째 층의 가중치
        self.params['b2'] = np.zeros(output_size)   # 두번째 층의 편향
        
    # 실제 추론을 하는 곳
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        t = np.argmax(t, axis = 1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}    # 기울기를 보관하는 딕셔너리 변수 (numerical_gradient() 메서드의 반환 값)
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) # 각 층의 가중치와 편향의 기울기를 계산해서 저장한다.
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [11]:
# 만든 클래스는 prams, grads라는 인스턴스 변수로 갖는다.

net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)

print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [12]:
x = np.random.rand(100, 784) # 더미 데이터 (100장 분량)
y = net.predict(x)

In [13]:
x = np.random.rand(100, 784) # 더미 데이터 (100장 분량)
t = np.random.rand(100, 10) # 더미 데이터 (100장 분량)

grads = net.numerical_gradient(x, t) # 기울기 계산

print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


TwoLayerNet의 메서드에는 __init__으로 클래스를 먼저 초기화시킨다.  
가중치는 정규분포를 따라 난수로, 편향은 0으로 초기화 한다.  

#### 4.5.2 미니배치 학습 구현하기

In [15]:
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize = True, one_hot_label = True)

train_loss_list = []

# 하이퍼파라미터
iters_num = 10000 # 반복횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

KeyboardInterrupt: 

미니배치 크기는 100으로 정한다. 즉, 매번 60,000개의 훈련 데이터에서 임의로 100개의 데이터를 추려낸다. 그리고 그 미니배치를 대상으로 SGD를 수행해 매개변수르르 갱신한다.  
경사법에 의한 갱신 횟수를 10,000번으로 설정하고 갱신할 때마다 손실함수를 계산해서 배열에 추가한다.

#### 4.5.3 시험 데이터로 평가하기

손실함수의 값이 학습이 진행됨에 따라 서서히 내려가는 것을 확인할 수 있다. 이때 손실함수의 값이란 정확히는 미니배치에 대한 손실 함수 이다. 그런데 훈련데이터 이외의 데이터도 잘 인식하는지를 확인해야 한다. 오버피팅이 일어날 수 도 있기 때문이다.  
훈련 데이터에서만 제대로 작동하고 그렇지 않은 이미지는 식별하지 못하는 문제이다.  
네트워크의 범용 능력을 평가하기 위해 훈련 데이터에 사용되지 않은 데이터를 사용한다.  

본 코드에서는 1에폭당 훈련 데이터와 시험 데이터에 대한 정확도를 기록해본다.  
에폭이란 학습에서 훈련 데이터를 모두 소진했을 때의 횟수에 해당된다. 10,000개의 데이터를 100개의 미니배치로 학습할 경우 SGD를 100회 반복하면 훈련데이터를 모두 소진하여 100회가 1에폭이 된다.

In [ ]:
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize = True, one_hot_label = True)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

# 하이퍼파라미터
iters_num = 10000 # 반복횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | "
             + str(train_acc) + ", " + str(test_acc))

1 에폭마다 정확도를 계산해서 기록한다.  
만일 오버피팅이 일어나면 훈련 데이터에 대해 정확해지나 시험 데이터의 정확도가 떨어지기 시작한다.  

### 4.6 정리

신경망은 손실 함수라는 지표를 통해 해당 값을 가장 작아지게 하는 가중치 매개변수를 찾아내는데 목표가 있다.  
이를 찾기 위해 다양한 경사법이 존재한다.  